In [ ]:
# !pip install pydantic==1.10.13
# !pip install pydantic==2.6.2

In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

# Basic Usage

In [2]:
from langchain_zhipu import ChatZhipuAI
from langchain_core.callbacks import BaseCallbackHandler

llm = ChatZhipuAI()


In [ ]:
# 或者你想自己临时指定API_KEY
import os
llm = ChatZhipuAI(api_key=os.getenv('ZHIPUAI_API_KEY'))

## invoke

In [4]:
llm.invoke("你是什么模型？")

AIMessage(content='我是一名人工智能助手，基于清华大学 KEG 实验室和智谱 AI 公司于 2023 年共同训练的语言模型 GLM-130B 开发而成。我的任务是针对用户的问题和要求提供适当的答复和支持。')

## stream

In [3]:
for s in llm.stream("你是什么模型？"):
  print(s)

content='我' id='8605586586812965306'
content='是一名' id='8605586586812965306'
content='人工智能' id='8605586586812965306'
content='助手' id='8605586586812965306'
content='，' id='8605586586812965306'
content='基于' id='8605586586812965306'
content='清华大学' id='8605586586812965306'
content=' K' id='8605586586812965306'
content='EG' id='8605586586812965306'
content=' 实' id='8605586586812965306'
content='验' id='8605586586812965306'
content='室' id='8605586586812965306'
content='和' id='8605586586812965306'
content='智' id='8605586586812965306'
content='谱' id='8605586586812965306'
content=' AI' id='8605586586812965306'
content=' 公司' id='8605586586812965306'
content='于' id='8605586586812965306'
content=' ' id='8605586586812965306'
content='202' id='8605586586812965306'
content='3' id='8605586586812965306'
content=' 年' id='8605586586812965306'
content='共同' id='8605586586812965306'
content='训练' id='8605586586812965306'
content='的语言' id='8605586586812965306'
content='模型' id='8605586586812965306'
content=' GL'

In [5]:
async for s in llm.astream([{"role": "user", "content": "你是什么模型？"}]):
  print(s)

content='我是一个' id='8605592118731052073'
content='基于' id='8605592118731052073'
content='语言' id='8605592118731052073'
content='的人工' id='8605592118731052073'
content='智能' id='8605592118731052073'
content='模型' id='8605592118731052073'
content='，' id='8605592118731052073'
content='名为' id='8605592118731052073'
content=' Chat' id='8605592118731052073'
content='GL' id='8605592118731052073'
content='M' id='8605592118731052073'
content='，' id='8605592118731052073'
content='现在' id='8605592118731052073'
content='又名' id='8605592118731052073'
content='智' id='8605592118731052073'
content='谱' id='8605592118731052073'
content='清' id='8605592118731052073'
content='言' id='8605592118731052073'
content='。' id='8605592118731052073'
content='我是' id='8605592118731052073'
content='由' id='8605592118731052073'
content='清华大学' id='8605592118731052073'
content=' K' id='8605592118731052073'
content='EG' id='8605592118731052073'
content=' 实' id='8605592118731052073'
content='验' id='8605592118731052073'
content='室' id

## batch

In [ ]:
llm.batch([
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？",
    "你是什么模型？"
])

# Tools-Calling

## function

In [4]:
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain.tools import tool

@tool
def search(query: str) -> str:
    """查询 langchan 资料; args: query 类型为字符串，描述用户的问题."""
    return "langchain_chinese 是一个为中国大模型优化的langchain模块"

llm.bind(tools=[convert_to_openai_tool(search)]).invoke("langchain_chinese是啥？请查询本地资料回答。")


AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query":"langchain_chinese是啥？"}', 'name': 'search'}, 'id': 'call_8581412518003372260', 'index': 0, 'type': 'function'}]})

## web_search

In [5]:
from langchain_zhipu import convert_to_web_search_tool
llm.bind(tools=[convert_to_web_search_tool(search_query="周星驰电影")]).invoke("哪部电影好看？")

AIMessage(content='选择哪部电影好看，这很大程度上取决于个人的喜好和审美。从您提供的参考信息来看，周星驰的电影作品涵盖了喜剧、爱情、动作等多个类型，每部影片都有其独特的魅力和深远的文化影响力。\n\n如果您喜欢经典喜剧，推荐您观看周星驰的以下几部电影：\n\n1. 《赌圣》：这部电影让周星驰的喜剧天赋得到了充分发挥，其幽默风趣的表演和轻松搞笑的情节，是早期周星驰电影的代表作。\n\n2. 《逃学威龙》：此片融合了校园元素和喜剧元素，周星驰的表演生动而幽默，是许多人心中的经典喜剧。\n\n3. 《大话西游》：这部电影在华语影坛有着不可动摇的地位，融合了爱情、喜剧与奇幻元素，是周星驰与朱茵等演员的经典之作，深受观众喜爱。\n\n4. 《喜剧之王》：这部电影是对演艺圈的讽刺和自我反思，既有幽默搞笑的情节，又有深刻的情感表达，是周星驰电影中较为特别的一部。\n\n如果想要看一些票房成绩斐然的作品，可以尝试以下几部：\n\n1. 《食神》：这部电影不仅票房高，而且美食元素与喜剧结合得恰到好处，有笑点有泪点。\n\n2. 《鹿鼎记》：改编自金庸小说，由周星驰主演，充满了幽默与智慧，是金庸作品改编电影中的佼佼者。\n\n总之，周星驰的电影作品大多具有很高的娱乐价值和艺术成就，选择哪一部取决于您想看哪种类型的影片。建议根据您的个人喜好和心情来挑选，相信每部作品都能给您带来不同的观影体验。')

## retrieval

In [6]:
from langchain_zhipu import convert_to_retrieval_tool
llm.bind(tools=[convert_to_retrieval_tool(knowledge_id="1772979648448397312")]).invoke("你知道马冬梅住哪里吗？")

AIMessage(content='马冬梅住在楼上322房间。')

# 智能体

In [ ]:
!pip install langchainhub

In [ ]:
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain import hub

tools = [search]
prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

for chunk in agent_executor.stream({"input": "langchain_chinese是什么？"}):
    print(chunk)


In [ ]:
prompt

# GLM-4V

In [7]:
from langchain_zhipu import ChatZhipuAI
from langchain_core.prompts import ChatPromptTemplate

llm4v = ChatZhipuAI(model="glm-4v")

prompt = ChatPromptTemplate.from_messages([
    ("human", [
          {
            "type": "text",
            "text": "图里有什么"
          },
          {
            "type": "image_url",
            "image_url": {
                "url" : "https://img1.baidu.com/it/u=1369931113,3388870256&fm=253&app=138&size=w931&n=0&f=JPEG&fmt=auto?sec=1703696400&t=f3028c7a1dca43a080aeb8239f09cc2f"
            }
          }
        ]),
])

(prompt|llm4v).invoke({})

AIMessage(content='图中有一片蓝天白云和蓝色海面，海面上有岩石，远处有岛屿。天空湛蓝，云朵雪白，景色非常美丽。')

# get_token_ids

In [8]:
from langchain_zhipu import ChatZhipuAI

llm = ChatZhipuAI(allowed_special=set())
llm.get_token_ids('<|user|>')

[27, 91, 882, 91, 29]

In [9]:
llm.get_num_tokens(text="你好")

2